This notebook reproduces Figure 9 in the manuscript: inversion of synthetic data from a nonlinear subglacial lake model for the basal vertical velocity.

The synthetic data are produced by running the default simulation in the repository: 
https://github.com/agstub/sglake-parallel

In [1]:
# add path to code
import sys
sys.path.insert(0, '../code')
import numpy as np

import aux 
aux.nonlin_ex = 1          # specify that we are importing the nonlinear example data/solution

import params
params.cg_tol = 1.5e-3

In [ ]:
from synthetic_data import make_data,make_fields
from main import main

noise_level = 0.01                           # noise level (scaled relative to elevation anomaly norm)

inv_w = 1                                    # turn basal velocity inversion 'on'
inv_beta = 0                                 # turn basal drag inversion 'off'
sol_true = make_fields(inv_w,inv_beta)       # get the "true" inverse solution
data = make_data(inv_w,inv_beta,noise_level) # create the data
eps_w = 5e-2                                 # set regularization parameter to ~optimal value

vel_locs = np.zeros(np.shape(data[0]),dtype=int)
sol,fwd,mis = main(data,vel_locs,inv_w,inv_beta,eps_beta=0,eps_w=eps_w);

Solving normal equations with CG....

CG iter. 10: rel. residual norm = 9.88e-02,  tol = 1.50e-03
CG iter. 20: rel. residual norm = 2.19e-02,  tol = 1.50e-03
CG iter. 30: rel. residual norm = 2.25e-02,  tol = 1.50e-03
CG iter. 40: rel. residual norm = 1.05e-02,  tol = 1.50e-03
CG iter. 50: rel. residual norm = 8.92e-03,  tol = 1.50e-03
CG iter. 60: rel. residual norm = 2.08e-02,  tol = 1.50e-03
CG iter. 70: rel. residual norm = 7.13e-03,  tol = 1.50e-03
CG iter. 80: rel. residual norm = 3.75e-03,  tol = 1.50e-03
CG iter. 90: rel. residual norm = 3.16e-03,  tol = 1.50e-03
CG iter. 100: rel. residual norm = 3.70e-03,  tol = 1.50e-03
CG iter. 110: rel. residual norm = 4.72e-03,  tol = 1.50e-03
CG iter. 120: rel. residual norm = 7.46e-03,  tol = 1.50e-03
CG iter. 130: rel. residual norm = 2.60e-03,  tol = 1.50e-03
CG iter. 140: rel. residual norm = 6.37e-03,  tol = 1.50e-03
CG iter. 150: rel. residual norm = 4.77e-03,  tol = 1.50e-03
CG iter. 160: rel. residual norm = 3.03e-03,  tol = 1.50

Now we calculate the true and estimated volume change $\Delta V = V-V_0$ from the basal vertical velocity:

In [ ]:
from scipy.integrate import trapezoid,cumulative_trapezoid
from params import dx,dt
wb = sol[:,:,50]
wb_true = sol_true[:,:,50]
dV_inv = cumulative_trapezoid(trapezoid(wb,dx=dx,axis=-1),dx=dt,initial=0)
dV_true = cumulative_trapezoid(trapezoid(wb_true,dx=dx,axis=-1),dx=dt,initial=0)

Finally, we can plot the solution:

In [ ]:
from plotting import snapshots,plot_movie
snapshots(data,fwd,sol,sol_true,inv_w,inv_beta,dV_inv,dV_true)
#plot_movie(data,fwd,sol,sol_true,inv_w,inv_beta)